In [ ]:
import IPython.display as ipd
import torch
import matplotlib.pyplot as plt
import scipy.io
import numpy as np
import torch.nn.functional as F
import librosa.output
from matplotlib import cm
import matplotlib
import mir_eval
from sdr import GetSDR, compute_measures

In [ ]:
matplotlib.style.use('ggplot')
np.random.seed(123)
SAMPLE_RATE=22050

In [ ]:
from data import SpatialAudioDatasetWaveform, RealDataset, MixedDataset
from network import center_trim, Demucs

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
# fg_file = '/projects/grail/vjayaram/DinTaiFung/dataset_generators/harry_potter_10min_16b.wav'
#data_dir_real = '/projects/grail/audiovisual/datasets/DinTaiFung/2mics_real_vctk_recorded/test/'
#data_dir = '/projects/grail/audiovisual/datasets/DinTaiFung/2mics_real_samples_synthetic_composite/train/'
#data_test = RealDataset(data_dir_real, sr=SAMPLE_RATE, short_data=True)

data_dir_synth = "/projects/grail/audiovisual/datasets/DinTaiFung/pyroom_6mics_multiplevoices/test"
data_test = SpatialAudioDatasetWaveform(data_dir_synth, n_mics=6, sr=SAMPLE_RATE, target_fg_std=None, target_bg_std=None)

#data_test = MixedDataset(data_dir_synth, data_dir_real, 1000, sr=SAMPLE_RATE, perturb_prob=0.0, short_data=True)
device = torch.device('cuda:0')

In [ ]:
kwargs = {
    'num_workers': 0,
    'pin_memory': True
}

In [ ]:
import torch
print(torch.rand(1, device="cuda"))

In [ ]:
test_loader = torch.utils.data.DataLoader(data_test,
                                          batch_size=32,
                                          shuffle=False,
                                          **kwargs)

In [ ]:
%ls checkpoints/90degrees_6mics/

In [ ]:
model = Demucs(sources=2, n_audio_channels=6)

In [ ]:
model.load_state_dict(torch.load('checkpoints/90degrees_6mics/90degrees_6mics_1.pt'))
#model.load_state_dict(torch.load('checkpoints/vctk_real_recording_22050_mixed/vctk_real_recording_22050_mixed_2.pt'))
#model.load_state_dict(torch.load('checkpoints/vctk_real_recording_3200_new_mics/vctk_real_recording_3200_new_mics_24.pt'))
#model.load_state_dict(torch.load('checkpoints/pyroom_synthetic_3voices_22050/pyroom_synthetic_3voices_22050_22.pt'))
# model.load_state_dict(torch.load('checkpoints/vctk_real_recording_22050_multiplevoices/vctk_real_recording_22050_multiplevoices_7.pt'))
model.train = False
model.to(device)

In [ ]:
np.random.seed(123)
with torch.no_grad():
    #for batch_idx, (type_name, (data, label_voice_signals)) in enumerate(test_loader):
    for batch_idx, (data, label_voice_signals) in enumerate(test_loader):
        import pdb
        pdb.set_trace()
        print(batch_idx)
        if batch_idx != 0: continue
#         data = label_bg_signals[:,0,:,:]
        data = data.to(device)
        label_voice_signals = label_voice_signals.to(device)

        # Normalize input
        data_transformed = (data * 2**15).round() / 2**15
        ref = data_transformed.mean(0)
        print("Ref mean {}, ref std {}".format(ref.mean(), ref.std()))
        # ref_mean = -0.00035191531060263515  # From synthetic dataset
        # ref_std = 0.044271234422922134
        data_transformed = (data_transformed - ref.mean()) / (ref.std())

        # Run through the model
        valid_length = model.valid_length(data_transformed.shape[-1])
        delta = valid_length - data_transformed.shape[-1]
        padded = F.pad(data_transformed, (delta // 2, delta - delta // 2))

        output_signal, output_locs = model(padded)
        output_signal = center_trim(output_signal, data_transformed)
        output_locs = center_trim(output_locs, data_transformed)
        
        output_signal = output_signal * ref.std() + ref.mean()
        output_voices = output_signal[:, 0]

        break

In [ ]:
print(output_voices.shape)
print(label_voice_signals.shape)
for i in range(32):
    print("----------------")
    #print(type_name[i])
    print("idx {} loss {}".format(i, model.voice_loss(output_voices[i], label_voice_signals[i, 0])[0]))
    print(label_voice_signals[i,:,:].cpu().detach().numpy().std())
    
    #import pdb
    #pdb.set_trace()
    #input_sdr, input_sir, input_sar, input_perm = mir_eval.separation.bss_eval_sources(label_voice_signals[i, 0].cpu().numpy(), data[i].cpu().numpy(), compute_permutation=False)
    #output_sdr, output_sir, output_sar, output_perm = mir_eval.separation.bss_eval_sources(label_voice_signals[i, 0].cpu().numpy(), output_voices[i].cpu().numpy(), compute_permutation=False)
    #print("idx {} input SDR {} output SDR {}".format(i, input_sdr, output_sdr))

In [ ]:
idx = 21
inputs = data[idx,:,:].cpu().detach().numpy()

In [ ]:
label_voices = label_voice_signals[idx,:,:].cpu().detach().numpy()

In [ ]:
output_voice_signals = output_voices[idx,:,:].cpu().detach().numpy()
print(output_voice_signals.shape)

In [ ]:
ipd.Audio(data=(inputs[0]), rate=SAMPLE_RATE, normalize=False)

In [ ]:
ipd.Audio(data=(label_voices[0]), rate=SAMPLE_RATE, normalize=False)

In [ ]:
ipd.Audio(data=output_voice_signals[0], rate=SAMPLE_RATE, normalize=False)

In [ ]:
def correlations(data):
    correlations = []
    for offset in range(-100, 100, 1):
        correlation = np.mean((np.roll(data[0], offset) - data[1])**2)
        correlations.append(correlation)
    return np.argmin(correlations)

In [ ]:
ipd.Audio(data=(output_bg_signals[0]), rate=22050, normalize=True)

In [ ]:
from librosa.core import stft, istft
from librosa.display import specshow

In [ ]:
y = inputs[0]

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=1.0)
plt.figure(figsize=(16, 8), dpi=300)
plt.subplot(1, 2, 1)
specshow(D, y_axis='linear', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.subplot(1, 2, 2)
specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
plt.plot()

In [ ]:
y = output_voice_signals.mean(0)

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=1.0)
plt.figure(figsize=(16, 8), dpi=300)
plt.subplot(1, 2, 1)
specshow(D, y_axis='linear', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.subplot(1, 2, 2)
specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
plt.plot()

In [ ]:
y = label_voices[0][0]

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=1.0)
plt.figure(figsize=(16, 8), dpi=300)
plt.subplot(1, 2, 1)
specshow(D, y_axis='linear', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.subplot(1, 2, 2)
specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
plt.plot()

In [ ]:
plt.figure(figsize=(8,6), dpi=300)
counts, bins = np.histogram(D, bins=30)
plt.hist(bins[:-1], bins, weights=counts)
plt.ylim([1, 1e5])
plt.yscale('log')
plt.ylabel('count')
plt.xlabel('dB')
plt.title('Histogram')
plt.plot()

In [ ]:
Y = stft(y)
plt.plot(np.sum(np.abs(Y), axis=1))

In [ ]:
Y[:2, :] = 0
plt.plot(np.sum(np.abs(Y), axis=1))

In [ ]:
Y[np.where(np.sum(np.abs(Y), axis=1) >= 10), :] = 0
plt.plot(np.sum(np.abs(Y), axis=1))

In [ ]:
# Y = librosa.util.normalize(Y)

In [ ]:
plt.plot(istft(Y))

In [ ]:
D = librosa.amplitude_to_db(np.abs(Y), ref=1.0)
librosa.display.specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')

In [ ]:
newy = istft(Y)

In [ ]:
D = librosa.amplitude_to_db(np.abs(librosa.stft(newy)), ref=1.0)
plt.figure(figsize=(16, 8), dpi=300)
plt.subplot(1, 2, 1)
librosa.display.specshow(D, y_axis='linear', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.subplot(1, 2, 2)
librosa.display.specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
plt.plot()

In [ ]:
plt.figure(figsize=(8,6), dpi=300)
counts, bins = np.histogram(D, bins=30)
plt.hist(bins[:-1], bins, weights=counts)
plt.ylim([1, 1e5])
plt.yscale('log')
plt.ylabel('count')
plt.xlabel('dB')
plt.title('Histogram')
plt.plot()

In [ ]:
y_gt = label_voices[0]
D = librosa.amplitude_to_db(np.abs(librosa.stft(y_gt)), ref=1.0)
plt.figure(figsize=(16, 8), dpi=300)
plt.subplot(1, 2, 1)
librosa.display.specshow(D, y_axis='linear', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Linear-frequency power spectrogram')
plt.subplot(1, 2, 2)
librosa.display.specshow(D, y_axis='log', cmap=cm.jet)
plt.colorbar(format='%+2.0f dB')
plt.title('Log-frequency power spectrogram')
plt.plot()

In [ ]:
plt.figure(figsize=(8,6), dpi=300)
counts, bins = np.histogram(D, bins=30)
plt.hist(bins[:-1], bins, weights=counts)
plt.ylim([1, 1e5])
plt.yscale('log')
plt.ylabel('count')
plt.xlabel('dB')
plt.title('Histogram')
plt.plot()

In [ ]:
x = data[0][1].cpu().numpy()

In [ ]:
ipd.Audio(data=(newy), rate=22050)

In [ ]:
ipd.Audio(data=(labels[3]), rate=22050)

In [ ]:
ipd.Audio(data=(newy), rate=22050)

In [ ]:
top = np.sum(librosa.util.normalize(outputs[0]) ** 2) / outputs[0].shape[0]
bottom = np.sum(librosa.util.normalize(labels[0]) ** 2) / labels[0].shape[0]

In [ ]:
SDR = 10 * np.log10(top / bottom)

In [ ]:
print(SDR)

In [ ]:
binaural = False
if outputs.shape[0] == 2:
    binaural = True

In [ ]:
import os
base_dir = 'jan24'
os.makedirs(base_dir, exist_ok=True)

In [ ]:
if not binaural:
    for i in range(outputs.shape[0]):
        librosa.output.write_wav('{}/input_{}.wav'.format(base_dir, i), inputs[i], sr=22050)
        librosa.output.write_wav('{}/output_{}.wav'.format(base_dir, i), 100 * outputs[i], sr=22050)
        librosa.output.write_wav('{}/label_{}.wav'.format(base_dir, i), labels[i], sr=22050)
else:
    librosa.output.write_wav('{}/input.wav'.format(base_dir, i), np.asfortranarray(np.vstack(inputs)), sr=22050)
    librosa.output.write_wav('{}/output.wav'.format(base_dir, i), np.asfortranarray(np.vstack(outputs)), sr=22050)
    librosa.output.write_wav('{}/label.wav'.format(base_dir, i), np.asfortranarray(np.vstack(labels)), sr=22050)
        

233 => 1.121
150 => 1.835
100 => 1.036
50 => 0.648